<a href="https://colab.research.google.com/github/DLPY/Regression-Session-2/blob/master/Regression_Session_2_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# House Price Prediction based on Postal Code, Number of Bathrooms, Car Parking and Property Type

Detail on Data: https://www.kaggle.com/mihirhalai/sydney-house-prices

## **1.Import necessary packages for performing EDA and Regression**

In [ ]:
import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np 
import seaborn as sns
from sklearn.linear_model import (Ridge, Lasso, ElasticNet, RidgeCV, LassoCV, ElasticNetCV)
from sklearn.metrics import (r2_score, mean_squared_error)
from sklearn.model_selection import (RepeatedKFold, train_test_split)
from sklearn.preprocessing import StandardScaler

%matplotlib inline

pd.set_option('display.max_colwidth', None)

## **2.Read data from csv file into Pandas dataframe**

In [ ]:
# CSV is first read in from a github raw file another option is to import the notebook to your session storage by click on the file icon on left toolbar then importing csv
!wget https://raw.githubusercontent.com/DLPY/Regression-Session-2/master/Data/SydneyHousePrices.csv
df = pd.read_csv('SydneyHousePrices.csv')

## **3.Exploratory Data Analysis (EDA)**
Perform data analysis, cleaning and transformation.

### i) Data Analysis

In [ ]:
# Display the count of rows and columns.
df.shape

In [ ]:
# Review a small sample of the data.
df.head(5)

In [ ]:
# Review the data types.
df.dtypes

### From the above, notice the different types of data: integer (int64), float (float64), and text (object).
---

### Review the date range.

Additional date-aggregation functions become available when the data type is converted to date format.  

The data type for date range is currently text, which is okay for now.

In [ ]:
print('Date ranges from {} to {}'.format(df.Date.min(), df.Date.max()))

### Review descriptive statistics of the numerical data.

In [ ]:
round(df.describe(), 2)

##### From the above max (bed & bath) and min (sellPrice) - It shows the dataset has outliers that need to be removed.
---

In [ ]:
# Detailed overview of the dataframe itself.
df.info()

##### From the above, notice that the Non-Null Count values are different for bed and car.
---

### ii) Data Cleaning

In [ ]:
df.isnull().sum()

### Aggregate the missing values by date
Combining the method from above with the dataframe's groupby method to group missing car/bed values by date.

Plotting the output with the dataframe's plot method to see if there are any pockets of missing data.

In [ ]:
df.groupby(['Date'])['car'].apply(lambda x: x.isnull().sum()).plot()

In [ ]:
df.groupby(['Date'])['bed'].apply(lambda x: x.isnull().sum()).plot()

### Fill in the missing values
Assuming that houses in a particular area have a similar bath count, missing values can be filled in by:
 1. Grouping postal code and bath.
 2. Calculating the median value of bed and car for each of these groups.
 3. Filling in the missing values with the median count of each group.

In [ ]:
df['bed'].fillna(df.groupby(['postalCode', 'bath'])['bed'].transform('median'), inplace=True)

In [ ]:
df['car'].fillna(df.groupby(['postalCode', 'bath'])['car'].transform('median'), inplace=True)

### Remove outliers in the data
Using a function that removes the outliers from each column on the list, loop through a list of specific columns that were identified earlier in the EDA as having outliers.

The resulting dataframe does not contain any outliers.

In [ ]:
def remove_outlier(df_in, col_name):
    '''Removes outliers from a specified column of a dataframe using IQR and returns an updated dataframe.'''
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3 - q1 # Interquartile range
    fence_low  = q1 - 1.5 * iqr
    fence_high = q3 + 1.5 * iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out

In [ ]:
outliers = ['bath', 'sellPrice', 'car', 'bed']

for outlier in outliers:
    df = remove_outlier(df, outlier).reset_index(drop=True)

### iii) Data Transformation

The model must include some date related features in order to make better predictions.

New features can be created using the purchase dates.

 * Recent house prices are typically different from historic prices.
   - The date range of the model should be fairly recent, so date can be filtered to only include three years of the most recent data.
 * House prices tend to move slowly, on a monthly basis. 
   - A 'diffDate' feature can be created by calculating the difference of sale date and most recent date within the data.
 * There may be annual seasonality associated with house purchases.
   - A 'Quarter' feature can be created that bins the dates by annual quarter.

Transform dates for analysis of sales type prices of property types by year, month.

In [ ]:
# Convert the 'Date' datatype to datetime so that Pandas date functions become available.
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
# Get the maximum purchase date.
max_date = df['Date'].max()
max_date

In [ ]:
# Filter the dataframe so that the most recent data is available, three years in this case.
df = df[df['Date'] >= (max_date - np.timedelta64(3, 'Y'))]

In [ ]:
df.shape

In [ ]:
# Create the 'diffDate' feature.
df['diffDate'] = df['Date'].apply(lambda x: max_date - x)

In [ ]:
# Alter 'diffDate' so that it captures the timeframe values as months.
df['diffDate'] = df['diffDate'] / np.timedelta64(1, 'M')

In [ ]:
# Alter 'diffDate' so that monthly values are ints, not floats. Floats would be equivalent to weekly values, too granular.
df['diffDate'] = df['diffDate'].astype(int)

In [ ]:
# The Pandas date type allows the date to be split in various ways: year, month, day, quarter.
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Quarter'] = df['Date'].dt.quarter

In [ ]:
# Plotting sellPrice by year
sns.pointplot(data=df, x='Year', y='sellPrice', hue='propType', ci=None)
plt.title('Sale Prices by Year')
plt.legend(title='Property Type', bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

In [ ]:
# Plotting sellPrice by quarter.
sns.pointplot(data=df, x='Quarter', y='sellPrice', hue='propType', ci=None)
plt.title('Sale Prices by Quarter')
plt.legend(title='Property Type', bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

##### From the above, notice the overall seasonal and annual trends of each property type.
---

Create a new dataframe that captures the median sale price value of similar sized property types within each postal code and suburb.

In [ ]:
medSellPrice = df.groupby(['postalCode', 'suburb', 'bath', 'car', 'bed'])['sellPrice'].apply(lambda x: x.median()).reset_index()

In [ ]:
# Rename the column as 'medSellPrice' - median sell price
medSellPrice = medSellPrice.rename(columns={'sellPrice': 'medSellPrice'})
medSellPrice[:5]

##### From the above, notice the median sale price for each property size are different even when the postal code and suburb are the same.
---

In [ ]:
# Merge the dataframes based on property similarity, postalCode and sellPrice
df = pd.merge(df, medSellPrice, how='outer', on=['postalCode', 'suburb', 'bath', 'car', 'bed'])

In [ ]:
df.head()

Encoding the categorical variables - Change the text into numbers

Review the unique values within property type.

In [ ]:
df.propType.unique()

In [ ]:
# Drop warehouse, acreage, and other property types in order to focus specifically on housing data.
df = df.drop(df[(df.propType == 'warehouse') | (df.propType == 'acreage') | (df.propType == 'other')].index)

Convert the property type values into numeric categorical labels so that this data can be used in the model.

In [ ]:
df['propType'] = df['propType'].astype('category').cat.codes

In [ ]:
df.head()

##### From the above, notice that:
 * diffDate is a numeric value representing the approximate count of months from sale date to July 6, 2019, (the max date in the data).
 * The original date column has been split into new columns: Year, Month, Day, Quarter, diffDate, and medSellPrice.
 * The propType categories have been converted to a numeric value. Warehouse, acreage, and other property types have been removed to specifically focus on housing.
 ---

Quick review - columns that are assumed to be not useful and will be dropped:
* **Date** - diffDate will be used instead.
* **Id** - This is simply a row number of the data.
* **suburb** - postalCode will be used instead because it is a more generalised representation of locality.

## **4.Choosing predictors and target variables for performing Regression Analysis**
**Target and Source variables**

* **Target Variable:** sellingPrice
* **Predictor Variables:** ordDate, postalCode, bed, bath, car, propType

Create a new dataframe that includes only the selected columns

In [ ]:
df_new = df[['postalCode', 'bed', 'bath', 'car', 'propType', 'diffDate', 'Year', 'Month', 'Day', 'Quarter', 'medSellPrice', 'sellPrice']]

In [ ]:
df_new.head()

### Investigate correlation in the new dataframe.

Pandas has a built-in correlation function.

In [ ]:
corr = df_new.corr()

### Create a heatmap plot of the correlations, using a mask to hide redundant information and correct aspect ratio to ensure proper spacing of the chart.

In [ ]:
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
cmap = sns.diverging_palette(10, 250, as_cmap=True)
f, ax = plt.subplots(figsize=(11, 9))
ax.set_title('Correlation of Features')
sns.heatmap(corr, mask=mask, cmap=cmap, square=True,
            linewidths=0.2, cbar_kws={'shrink': 0.5}, ax=ax, annot=True)

From the above, notice:

The high positive correlation between
* Month / Quarter
* Sell Price / Median Sell Price
* Bed / Bath

The slight positive correlation between
* Bed / Bath / Median Sell Price / Sell Price
* Diffdate / Month / Quarter
* Bed / Bath / Car

The high negative correlation between
* Post Code / Sell Price / Median Sell Price

The slight negative correlation between
* Year / Month / Quarter
* Property Type / Bed

Multicollinearity has a negative impact on multiple regression models.  The steps for overcoming multicollinearity are different for the chosen model.

* **Training a model and testing on unseen data:** standardize the features for all models.
* **Use Lasso for feature selection:** regularization will be used for feature selection.

## **5.Create Final working dataset for Models**

In [ ]:
final_df = df[['postalCode', 'bed', 'bath', 'car', 'propType', 'diffDate', 'Year', 'Month', 'Day', 'Quarter', 'medSellPrice', 'sellPrice']]

In [ ]:
final_df.to_csv("Processed_SydneyHousePrices.csv", index=False)